# Bosch Production Line Performance
### Reduce manufacturing failures


A good chocolate soufflé is decadent, delicious, and delicate. But, it's a challenge to prepare. When you pull a disappointingly deflated dessert out of the oven, you instinctively retrace your steps to identify at what point you went wrong. Bosch, one of the world's leading manufacturing companies, has an imperative to ensure that the recipes for the production of its advanced mechanical components are of the highest quality and safety standards. Part of doing so is closely monitoring its parts as they progress through the manufacturing processes.

<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/5357/media/BoschManufacturingKaggleImage.jpg" style="max-height:400px; text-align:center;">

Because Bosch records data at every step along its assembly lines, they have the ability to apply advanced analytics to improve these manufacturing processes. However, the intricacies of the data and complexities of the production line pose problems for current methods.

In this competition, Bosch is challenging Kagglers to predict internal failures using thousands of measurements and tests made for each component along the assembly line. This would enable Bosch to bring quality products at lower costs to the end user.

### Evaluation

Submissions are evaluated on the Matthews correlation coefficient (MCC) between the predicted and the observed response. The MCC is given by:

<img src="https://latex.codecogs.com/gif.latex?MCC&space;=&space;\frac{(TP*TN)&space;-&space;(FP&space;*&space;FN)}{\sqrt{(TP&plus;FP)(TP&plus;FN)(TN&space;&plus;&space;FP)(TN&plus;FN)}}" title="MCC = \frac{(TP*TN) - (FP * FN)}{\sqrt{(TP+FP)(TP+FN)(TN + FP)(TN+FN)}}" />

where TP is the number of true positives, TN the number of true negatives, FP the number of false positives, and FN the number of false negatives.

### Description of Data

The data for this competition represents measurements of parts as they move through Bosch's production lines. Each part has a unique Id. The goal is to predict which parts will fail quality control (represented by a 'Response' = 1).

The dataset contains an extremely large number of anonymized features. Features are named according to a convention that tells you the production line, the station on the line, and a feature number. E.g. L3_S36_F3939 is a feature measured on line 3, station 36, and is feature number 3939.

On account of the large size of the dataset, we have separated the files by the type of feature they contain: numerical, categorical, and finally, a file with date features. The date features provide a timestamp for when each measurement was taken. Each date column ends in a number that corresponds to the previous feature number. E.g. the value of L0_S0_D1 is the time at which L0_S0_F0 was taken.

In addition to being one of the largest datasets (in terms of number of features) ever hosted on Kaggle, the ground truth for this competition is highly imbalanced. Together, these two attributes are expected to make this a challenging problem.

### File descriptions

* **train_numeric.csv** - the training set numeric features (this file contains the 'Response' variable)
* **test_numeric.csv** - the test set numeric features (you must predict the 'Response' for these Ids)
* **train_categorical.csv** - the training set categorical features
* **test_categorical.csv** - the test set categorical features
* **train_date.csv** - the training set date features
* **test_date.csv** - the test set date features
* **sample_submission.csv** - a sample submission file in the correct format

## Notebook Objective: Reviewing the Data

In any process involving data, the first goal should always be understanding the data. This involves looking at the data and answering a range of questions including (but not limited to):

* What **features** (columns) does the dataset contain?
* **How many records** (rows) have been provided?
* **What kind** of features:
    * Date Format ?
    * Numerical Values ?
    * Categorical values ?
    * Ordinal Values ?
* Are there **missing values**?
* How do the different features **relate to each other**?

## 1. Import all the libraries

#### Parallel Computation

The complete dataset takes around **14.35 GB of CSV Files**, with 7.17GB for test data and 7.18GB for training data.

Unfortunately due to the heavy datastructure of pandas dataframe, the **64GB of RAM available in our machine is not sufficient** to load everything into memory.

Two of the possibilities could be to load the data with:
- **Dask Library**: https://dask.pydata.org/en/latest/
- **Spark Python Library - PySpark**: http://spark.apache.org/docs/2.1.0/api/python/index.html

After a few tests, PySpark appears to be more flexible and faster to deal with *[Sparse Matrices](https://en.wikipedia.org/wiki/Sparse_matrix)*. Additionally, Spark benefits from a wide support from the industry and thus could be easily integrated in a Data Pipeline.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

import pandas as pd
from collections import defaultdict

sc = SparkContext()
sqlContext = SQLContext(sc)

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

## 2. Load every csv file with Pandas

In [3]:
def printDf(sprkDF):
    newdf = sprkDF.limit(5).toPandas()
    return newdf.head()

In [4]:
def loadDf(filename):
    return sqlContext.read.load(
        filename, 
        format='com.databricks.spark.csv', 
        header='true', 
        inferSchema='true'
    )

### 2.0 Loading a small dummy file to check loading a dataframe with Dask

In [5]:
df_dummy = loadDf('data/dummy_data.csv')
printDf(df_dummy)

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,1,t1,t2,t3,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,5,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,8,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### 2.1 Load Sample Submission File

In [6]:
df_sample_submission = loadDf('data/sample_submission.csv')  
printDf(df_sample_submission)

,Id,Response
0,1,0
1,2,0
2,3,0
3,5,0
4,8,0


### 2.2 Load Training Numeric Features File

In [7]:
df_train_numeric = loadDf('data/train_numeric.csv')   
printDf(df_train_numeric)

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,None,None,None,None,None,None,None,None,None,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,None,None,None,None,None,None,None,None,None,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,None,None,None,None,None,None,None,None,None,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,None,None,None,None,None,None,None,None,None,0


### 2.3 Load Testing Numeric Features File

In [8]:
df_test_numeric = loadDf('data/test_numeric.csv')  
printDf(df_test_numeric)

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,5,-0.016,-0.026,-0.033,-0.016,0.205,-0.157,0.0,0.008,0.087,...,None,None,None,None,None,None,None,None,None,None
4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


### 2.4 Load Training Categorical Features File

In [ ]:
df_train_categorical = loadDf('data/train_categorical.csv')   
printDf(df_train_categorical)

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,6,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,7,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,9,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,11,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### 2.5 Load Testing Categorical Features File

In [ ]:
df_test_categorical = loadDf('data/test_categorical.csv')   
printDf(df_test_categorical)

### 2.6 Load Training Date Features File

In [ ]:
df_train_date = loadDf('data/train_date.csv')
printDf(df_train_date)

### 2.7 Load Testing Date Features File

In [ ]:
df_test_date = loadDf('data/test_date.csv')  
printDf(df_test_date)

## 3. Dataset Analysis
### 3.1 Dataset Size

In [ ]:
df_train_numeric_size = df_train_numeric.count()
df_train_categorical_size = df_train_categorical.count()
df_train_date_size = df_train_date.count()

df_test_numeric_size  = df_test_numeric.count()
df_test_categorical_size  = df_test_categorical.count()
df_test_date_size  = df_test_date.count()

In [ ]:
print("Number of lines - df_train_numeric: %s" % '{:,}'.format(df_train_numeric_size))
print("Number of lines - df_train_categorical: %s" % '{:,}'.format(df_train_categorical_size))
print("Number of lines - df_train_date: %s\n" % '{:,}'.format(df_train_date_size))

print("Number of lines - df_test_numeric: %s" % '{:,}'.format(df_test_numeric_size))
print("Number of lines - df_test_categorical: %s" % '{:,}'.format(df_test_categorical_size))
print("Number of lines - df_test_date: %s" % '{:,}'.format(df_test_date_size))

### 3.2 Column Analysis

#### 3.2.1 df_train_categorical

In [ ]:
result = defaultdict(list)
cols = df_train_categorical.columns

for col in log_progress(cols[1:], every=1): # We do not count for the "Id" Column
    
    arr = df_train_categorical.select(col).distinct().collect()

    for _ in arr:
        key = _.__fields__[0]
        val = _[key] if _[key] is not None else "No Value"
        result[col].append(val)

In [ ]:
df_column_analysis = pd.DataFrame.from_dict(result, orient='index', dtype='str')

writer = pd.ExcelWriter('PythonExport.xlsx')

df_column_analysis.to_excel(writer,'ColumnAnalysis')
writer.save()